In [1]:
import tensorflow_core as tf
import numpy as np
from BregmanToolkit.bregman import suite as st
import matplotlib.pyplot as plt

/home/howard/collaborative-recommender/notebook


In [13]:
# files operation within tensorflow

filenames = tf.train.match_filenames_once(
    "/home/howard/collaborative-recommender/data/*.wav"
)
count_num_files = tf.size(filenames)
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
filename, file_contents = reader.read(filename_queue)

In [14]:
# init session

sess = tf.Session()
init = (tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

(None, None)

In [15]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

In [16]:
def get_chromagram(sess):
    F = st.Chromagram(
        sess.run(filename).decode("utf-8"), nfft=16384, wfft=8192, nhop=2205
    )
    return F.X

In [17]:
def extract_feature_vector(sess, chroma_data):
    chroma = tf.placeholder(tf.float32)
    max_freqs = tf.argmax(chroma, 0)  # first division

    num_features, num_samples = np.shape(chroma_data)
    freq_vals = sess.run(max_freqs, feed_dict={chroma: chroma_data})
    hist, _ = np.histogram(freq_vals, bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    return hist.astype(float) / num_samples

In [18]:
def get_dataset(sess):
    num_files = sess.run(count_num_files)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    xs = []
    for _ in range(num_files):
        chroma_data = get_chromagram(sess)
        x = np.matrix(extract_feature_vector(sess, chroma_data))

        if len(xs) == 0: xs = x
        else: xs = np.vstack((xs, x))

    coord.request_stop()
    coord.join()

    return xs

In [19]:
k = 2
max_iterations = 100

In [20]:
def initial_centroids(X, k):
    # assign centroids label
    # 1st centroids -> cluster 0
    # 2nd centroids -> cluster 1

    return X[0:k, :]

In [24]:
def assign_cluster(X, centroids):
    expanded_vectors = tf.expand_dims(X, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    
    print(expanded_vectors.get_shape())
    print(expanded_centroids.get_shape())

    # calculate distance from each vector to each centroids 
    distances = tf.reduce_sum(tf.square(tf.subtract(expanded_vectors,
        expanded_centroids)), 2)

    return tf.argmin(distances, 0)

In [25]:
def recenter_centroids(X, Y, k):
    sums = tf.unsorted_segment_sum(X, Y, k)
    counts = tf.unsorted_segment_sum(tf.ones_like(X), Y, k)

    return sums / counts    

In [29]:
X = get_dataset(sess)
# print(X)
centroids = initial_centroids(X, k)

i, coveraged = 0, False
while not coveraged and i < max_iterations:
    i += 1
    Y = sess.run(assign_cluster(X, centroids))
    centroids = sess.run(recenter_centroids(X, Y, k))

print(centroids)

(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)
(2, 1, 12)
(1, 1, 12)